# Workflow for a multi-regional energy system

In this application of the FINE framework, a multi-regional energy system is modeled and optimized.

All classes which are available to the user are utilized and examples of the selection of different parameters within these classes are given.

The workflow is structures as follows:
1. Required packages are imported and the input data path is set
2. An energy system model instance is created
3. Commodity sources are added to the energy system model
4. Commodity conversion components are added to the energy system model
5. Commodity storages are added to the energy system model
6. Commodity transmission components are added to the energy system model
7. Commodity sinks are added to the energy system model
8. The energy system model is optimized



# 1. Import required packages and set input data path

The FINE framework is imported which provides the required classes and functions for modeling the energy system. The pandas packages is used for import location and time series specific data. The os package is used for setting the input data path.

In [1]:
import FINE as fn
import pandas as pd
import os

%load_ext autoreload
%autoreload 2

In [2]:
cwd = os.getcwd()
inputDataPath = os.path.join(cwd,"InputData")

# 2. Create an energy system model instance 

The structure of the energy system model is given by the considered locations, commodities, the number of time steps as well as the hours per time step.

The commodities are specified by a unit (i.e. 'GW_electric', 'GW_H2lowerHeatingValue', 'Mio. t CO2/h') which can be given as an energy or mass unit per hour. Furthermore, the cost unit and length unit are specified.

In [3]:
locations = {'cluster_0','cluster_1','cluster_2','cluster_3','cluster_4','cluster_5','cluster_6','cluster_7'}
commodities = {'electricity','hydrogen','methane','biogas','CO2'}
numberOfTimeSteps=8760
hoursPerTimeStep=1

In [4]:
esM = fn.EnergySystemModel(locations=locations, commodities=commodities, numberOftimeSteps=8760,
                           hoursPerTimeStep=1, powerUnit='GW', energyUnit='GWh',
                           costUnit='1e9 Euro', lengthUnit='km')

In [5]:
CO2_reductionTarget = 1

# 3. Add commodity sources to the energy system model

## 3.1. Electricity sources

### Wind onshore

In [6]:
capacityMax=pd.read_excel(os.path.join(inputDataPath,'SpatialData','Wind','maxCapacityOnshore_GW_el.xlsx'),
                          index_col=0, squeeze=True)
operationRateMax=pd.read_excel(os.path.join(inputDataPath,'SpatialData','Wind','maxOperationRateOnshore_el.xlsx'))

esM.add(fn.Source(esM=esM, name='Wind (onshore)', commodity='electricity', hasDesignDimensionVariables=True,
                  operationRateMax=operationRateMax, capacityMax=capacityMax,
                  capexPerDesignDimension=1.1, opexPerDesignDimension=1.1*0.02, interestRate=0.08,
                  economicLifetime=20))

The locationalEligibility of a component was set based on the given fixed/maximum capacity of the component.


### Wind offshore

In [7]:
capacityMax=pd.read_excel(os.path.join(inputDataPath,'SpatialData','Wind','maxCapacityOffshore_GW_el.xlsx'),
                          index_col=0, squeeze=True)
operationRateMax=pd.read_excel(os.path.join(inputDataPath,'SpatialData','Wind','maxOperationRateOffshore_el.xlsx'))

esM.add(fn.Source(esM=esM, name='Wind (offshore)', commodity='electricity', hasDesignDimensionVariables=True,
                  operationRateMax=operationRateMax, capacityMax=capacityMax,
                  capexPerDesignDimension=2.3, opexPerDesignDimension=2.3*0.02, interestRate=0.08,
                  economicLifetime=20))

The locationalEligibility of a component was set based on the given fixed/maximum capacity of the component.


### PV

In [8]:
capacityMax=pd.read_excel(os.path.join(inputDataPath,'SpatialData','PV','maxCapacityPV_GW_el.xlsx'),
                          index_col=0, squeeze=True)
operationRateMax=pd.read_excel(os.path.join(inputDataPath,'SpatialData','PV','maxOperationRatePV_el.xlsx'))

esM.add(fn.Source(esM=esM, name='PV', commodity='electricity', hasDesignDimensionVariables=True,
                  operationRateMax=operationRateMax, capacityMax=capacityMax,
                  capexPerDesignDimension=0.65, opexPerDesignDimension=0.65*0.02, interestRate=0.08,
                  economicLifetime=25))

The locationalEligibility of a component was set based on the given fixed/maximum capacity of the component.


### Exisisting run-of-river hydroelectricity plants

In [9]:
capacityFix=pd.read_excel(os.path.join(inputDataPath,'SpatialData','HydroPower','fixCapacityROR_GW_el.xlsx'),
                          index_col=0, squeeze=True)
operationRateFix=pd.read_excel(os.path.join(inputDataPath,'SpatialData','HydroPower',
                                            'fixOperationRateROR_GW_el.xlsx'))

esM.add(fn.Source(esM=esM, name='Existing run-of-river plants', commodity='electricity',
                  hasDesignDimensionVariables=True,
                  operationRateFix=operationRateFix, capacityFix=capacityFix,
                  capexPerDesignDimension=0, opexPerDesignDimension=0.208))

The locationalEligibility of a component was set based on the given fixed/maximum capacity of the component.


## 3.2. Methane (natural gas and biogas)

### Natural gas

In [10]:
esM.add(fn.Source(esM=esM, name='Natural gas purchase', commodity='methane',
                  hasDesignDimensionVariables=False, commodityCost=0.0331*1e-3))

The locationalEligibility of a component is set to 1 (eligible) for all locations.


### Biogas

In [11]:
operationRateMax=pd.read_excel(os.path.join(inputDataPath,'SpatialData','Biogas',
                                            'biogasPotential_GWh_biogas.xlsx'))

esM.add(fn.Source(esM=esM, name='Biogas purchase', commodity='biogas', operationRateMax=operationRateMax,
                  hasDesignDimensionVariables=False, commodityCost=0.05409*1e-3))

The locationalEligibility of a component was set based on the given operation time series of the component.


## 3.3 CO2

### CO2

The CO2 source is required for the case in which it can be mixed with conventional natural gas.

In [12]:
esM.add(fn.Source(esM=esM, name='CO2 from enviroment', commodity='CO2',
                  hasDesignDimensionVariables=False, commodityLimitID='CO2 limit', yearlyLimit=366*(1-CO2_reductionTarget)))

The locationalEligibility of a component is set to 1 (eligible) for all locations.


# 4. Add conversion components to the energy system model

### Biogas to methane

In [13]:
esM.add(fn.Conversion(esM=esM, name='Biogas to methane',
                      commodityConversionFactors={'biogas':-1, 'methane':1, 'CO2':-201*1e-6*0.625},
                      hasDesignDimensionVariables=False))

The locationalEligibility of a component is set to 1 (eligible) for all locations.


### Existing combined cycle gas turbine plants

In [14]:
capacityMax=pd.read_excel(os.path.join(inputDataPath,'SpatialData','NaturalGasPlants',
                                       'existingCombinedCycleGasTurbinePlantsCapacity_GW_el.xlsx'),
                          index_col=0, squeeze=True)

esM.add(fn.Conversion(esM=esM, name='Existing CCGT plants (methane)',
                      commodityConversionFactors={'electricity':1, 'methane':-1/0.625, 'CO2':201*1e-6/0.625},
                      hasDesignDimensionVariables=True, capacityMax=capacityMax,
                      capexPerDesignDimension=0, opexPerDesignDimension=0.021, interestRate=0.08,
                      economicLifetime=33))

The locationalEligibility of a component was set based on the given fixed/maximum capacity of the component.


### New combined cycle gas turbine plants for biogas

In [15]:
esM.add(fn.Conversion(esM=esM, name='New CCGT plants (biogas)',
                      commodityConversionFactors={'electricity':1, 'biogas':-1/0.635},
                      hasDesignDimensionVariables=True, 
                      capexPerDesignDimension=0.7, opexPerDesignDimension=0.021, interestRate=0.08,
                      economicLifetime=33))

The locationalEligibility of a component is set to 1 (eligible) for all locations.


### New combined cycly gas turbines for hydrogen

In [16]:
esM.add(fn.Conversion(esM=esM, name='New CCGT plants (hydrogen)',
                      commodityConversionFactors={'electricity':1, 'hydrogen':-1/0.6},
                      hasDesignDimensionVariables=True, 
                      capexPerDesignDimension=0.927, opexPerDesignDimension=0.021, interestRate=0.08,
                      economicLifetime=33))

The locationalEligibility of a component is set to 1 (eligible) for all locations.


### Electrolyzers

In [17]:
esM.add(fn.Conversion(esM=esM, name='Electroylzers',
                      commodityConversionFactors={'electricity':-1, 'hydrogen':0.7},
                      hasDesignDimensionVariables=True, 
                      capexPerDesignDimension=0.5, opexPerDesignDimension=0.5*0.025, interestRate=0.08,
                      economicLifetime=10))

The locationalEligibility of a component is set to 1 (eligible) for all locations.


# 5. Add commodity storages to the energy system model

## 5.1. Electricity storage

### Lithium ion batteries

The self discharge of a lithium ion battery is here described as 3% per month. The self discharge per hours is obtained using the equation (1-$\text{selfDischarge}_\text{hour})^{30*24\text{h}} = 1-\text{selfDischarge}_\text{month}$.

In [18]:
esM.add(fn.Storage(esM=esM, name='Li-ion batteries', commodity='electricity',
                   hasDesignDimensionVariables=True, chargeEfficiency=0.95,
                   cyclicLifetime=10000, dischargeEfficiency=0.95, selfDischarge=1-(1-0.03)**(1/(30*24)),
                   chargeRate=1, dischargeRate=1,
                   capexPerDesignDimension=0.151, opexPerDesignDimension=0.002, interestRate=0.08,
                   economicLifetime=22))

The locationalEligibility of a component is set to 1 (eligible) for all locations.


## 5.2. Hydrogen storage

### Hydrogen filled salt caverns
The maximum capacity is here obtained by: dividing the given capacity (which is given for methane) by the lower heating value of methane and then multiplying it with the lower heating value of hydrogen.

In [19]:
capacityMax=pd.read_excel(os.path.join(inputDataPath,'SpatialData','GeologicalStorage',
                                       'existingSaltCavernsCapacity_GWh_methane.xlsx'),
                          index_col=0, squeeze=True)*3/10

esM.add(fn.Storage(esM=esM, name='Salt caverns (hydrogen)', commodity='hydrogen',
                   hasDesignDimensionVariables=True, designDimensionVariableDomain='discrete',
                   capacityPerUnit=133,
                   chargeRate=1/470.37, dischargeRate=1/470.37, sharedPotentialID='Existing salt caverns',
                   stateOfChargeMin=0.33, stateOfChargeMax=1, capacityMax=capacityMax,
                   capexPerDesignDimension=0.00011, opexPerDesignDimension=0.00057, interestRate=0.08,
                   economicLifetime=30))

The locationalEligibility of a component was set based on the given fixed/maximum capacity of the component.


## 5.3. Methane storage

### Methane filled salt caverns

In [20]:
capacityMax=pd.read_excel(os.path.join(inputDataPath,'SpatialData','GeologicalStorage',
                                       'existingSaltCavernsCapacity_GWh_methane.xlsx'),
                          index_col=0, squeeze=True)

esM.add(fn.Storage(esM=esM, name='Salt caverns (methane)', commodity='methane',
                   hasDesignDimensionVariables=True, designDimensionVariableDomain='discrete',
                   capacityPerUnit=443,
                   chargeRate=1/470.37, dischargeRate=1/470.37, sharedPotentialID='Existing salt caverns',
                   stateOfChargeMin=0.33, stateOfChargeMax=1, capacityMax=capacityMax,
                   capexPerDesignDimension=0.00004, opexPerDesignDimension=0.00001, interestRate=0.08,
                   economicLifetime=30))

The locationalEligibility of a component was set based on the given fixed/maximum capacity of the component.


## 5.4 Pumped hydro storage

### Pumped hydro storage

In [21]:
capacityFix=pd.read_excel(os.path.join(inputDataPath,'SpatialData','HydroPower',
                                       'fixCapacityPHS_storage_GWh_energyPHS.xlsx'),
                          index_col=0, squeeze=True)

esM.add(fn.Storage(esM=esM, name='Pumped hydro storage', commodity='electricity',
                   chargeEfficiency=0.88, dischargeEfficiency=0.88,
                   hasDesignDimensionVariables=True, selfDischarge=1-(1-0.00375)**(1/(30*24)),
                   chargeRate=0.16, dischargeRate=0.12, capacityFix=capacityFix,
                   capexPerDesignDimension=0, opexPerDesignDimension=0.000153))

The locationalEligibility of a component was set based on the given fixed/maximum capacity of the component.


# 6. Add commodity transmission components to the energy system model

## 6.1. Electricity transmission

### AC cables

In [22]:
capacityFix=pd.read_excel(os.path.join(inputDataPath,'SpatialData','ElectricGrid',
                                       'ACcableExistingCapacity_GW_el.xlsx'),
                          index_col=0, header=0)

esM.add(fn.Transmission(esM=esM, name='AC cables', commodity='electricity',
                        hasDesignDimensionVariables=True, capacityFix=capacityFix))

The distances of a component are set to a normalized values of 1.
The locationalEligibility of a component was set based on the given fixed/maximum capacity of the component.


### DC cables

In [23]:
capacityFix=pd.read_excel(os.path.join(inputDataPath,'SpatialData','ElectricGrid',
                                       'DCcableExistingCapacity_GW_el.xlsx'),
                          index_col=0, header=0)
distances=pd.read_excel(os.path.join(inputDataPath,'SpatialData','ElectricGrid',
                                       'DCcableLength_km.xlsx'),
                          index_col=0, header=0)
efficiency=pd.read_excel(os.path.join(inputDataPath,'SpatialData','ElectricGrid',
                                       'DCcableEfficiency.xlsx'),
                          index_col=0, header=0)

esM.add(fn.Transmission(esM=esM, name='DC cables', commodity='electricity', efficiency=efficiency,
                        hasDesignDimensionVariables=True, capacityFix=capacityFix))

The distances of a component are set to a normalized values of 1.
The locationalEligibility of a component was set based on the given fixed/maximum capacity of the component.


## 6.2 Methane transmission

### Methane pipeline

In [24]:
eligibility=pd.read_excel(os.path.join(inputDataPath,'SpatialData','Pipelines',
                                       'pipelineIncidence.xlsx'), index_col=0, header=0)
distances=pd.read_excel(os.path.join(inputDataPath,'SpatialData','Pipelines', 'pipelineIncidence.xlsx'),
                          index_col=0, header=0)

esM.add(fn.Transmission(esM=esM, name='Pipelines (methane)', commodity='methane', distances=distances,
                        hasDesignDimensionVariables=True, hasDesignDecisionVariables=True, bigM=300,
                        locationalEligibility=eligibility,
                        capexPerDesignDimension=0.000037, capexForDesignDecision=0.000314,
                        interestRate=0.08, economicLifetime=40))

## 6.3 Hydrogen transmission

### Hydrogen pipelines

In [25]:
eligibility=pd.read_excel(os.path.join(inputDataPath,'SpatialData','Pipelines', 'pipelineIncidence.xlsx'),
                          index_col=0, header=0)
distances=pd.read_excel(os.path.join(inputDataPath,'SpatialData','Pipelines', 'pipelineIncidence.xlsx'),
                          index_col=0, header=0)

esM.add(fn.Transmission(esM=esM, name='Pipeline (hydrogen)', commodity='hydrogen', distances=distances,
                        hasDesignDimensionVariables=True, hasDesignDecisionVariables=True, bigM=300,
                        locationalEligibility=eligibility,
                        capexPerDesignDimension=0.000177, capexForDesignDecision=0.00033,
                        interestRate=0.08, economicLifetime=40))

# 7. Add commodity sinks to the energy system model

## 7.1. Electricity sinks

### Electricity demand

In [26]:
operationRateFix=pd.read_excel(os.path.join(inputDataPath,'SpatialData','Demands',
                                            'electricityDemand_GWh_el.xlsx'))

esM.add(fn.Sink(esM=esM, name='Electricity demand', commodity='electricity',
                hasDesignDimensionVariables=False, operationRateFix=operationRateFix))

The locationalEligibility of a component was set based on the given operation time series of the component.


## 7.2. Hydrogen sinks

### Fuel cell electric vehicle (FCEV) demand

In [27]:
FCEV_penetration=0.5
operationRateFix=pd.read_excel(os.path.join(inputDataPath,'SpatialData','Demands',
                                            'hydrogenDemand_GWh_hydrogen.xlsx'))*FCEV_penetration

esM.add(fn.Sink(esM=esM, name='Hydrogen demand', commodity='hydrogen',
                  hasDesignDimensionVariables=False, operationRateFix=operationRateFix))

The locationalEligibility of a component was set based on the given operation time series of the component.


## 7.3. CO2 sinks

### CO2 exiting the system's boundary

In [28]:
esM.add(fn.Sink(esM=esM, name='CO2 to enviroment', commodity='CO2',
                hasDesignDimensionVariables=False, commodityLimitID='CO2 limit', yearlyLimit=366*(1-CO2_reductionTarget)))

The locationalEligibility of a component is set to 1 (eligible) for all locations.


# 8. Optimize energy system model

In [29]:
esM.cluster(numberOfTypicalPeriods=7)
esM.optimize(timeSeriesAggregation=True,
             optimizationSpecs='LogToConsole=1 OptimalityTol=1e-3 cuts=0 method=2 BarHomogeneous=1')

Declaring sets, variables and constraints for SourceSinkModeling
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.7709 sec)
Declaring sets, variables and constraints for TransmissionModeling
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.5579 sec)
Declaring sets, variables and constraints for StorageModeling
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(13.6086 sec)
Declaring sets, variables and constraints for ConversionModeling
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.2390 sec)
Declaring shared potential constraint...
		(0.0010 sec)
Declaring commodity balances...
		(1.4419 sec)
Declaring objective function...
		(0.9229 sec)
Academic license - for non-commercial use only
Parameter LogToConsole unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter OptimalityTol to 0.001
   Prev: 1e-06  Min: 1e-09  Max: 0.01  Default: 1e-06
Parameter Threads 

In [30]:
df = pd.DataFrame(esM._pyM.cap_conv.get_values(), index=[0]).T.swaplevel(i=0, j=1, axis=0).sort_index()
df = df.unstack(level=-1).fillna(0)
df.columns = df.columns.droplevel()
df

,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7
Electroylzers,21.942090,0.000000,0.000000,0.000000,12.535688,0.000000,0.000000,0.000000
Existing CCGT plants (methane),0.000000,0.000000,2.144325,0.000000,0.000000,0.878927,0.000000,0.000000
New CCGT plants (biogas),0.603867,0.545487,0.529770,0.834179,0.734495,0.798122,0.628174,0.561975
New CCGT plants (hydrogen),0.000000,0.000000,8.004811,3.527990,0.000000,0.000000,0.000000,0.000000


In [31]:
df = pd.DataFrame(esM._pyM.cap_srcSnk.get_values(), index=[0]).T.swaplevel(i=0, j=1, axis=0).sort_index()
df = df.unstack(level=-1).fillna(0)
df.columns = df.columns.droplevel()
df

,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7
Existing run-of-river plants,0.000000,0.544397,0.455005,0.037456,0.000000,0.043572,2.075538,0.655032
PV,17.970141,25.807194,18.698614,32.994484,20.883876,39.127909,22.626522,21.891252
Wind (offshore),16.107011,0.000000,0.000000,0.000000,28.616236,12.859400,0.000000,0.000000
Wind (onshore),58.169300,0.000000,22.323000,0.000000,54.560100,0.000000,39.382068,0.000000


In [32]:
df = pd.DataFrame(esM._pyM.cap_stor.get_values(), index=[0]).T.swaplevel(i=0, j=1, axis=0).sort_index()
df = df.unstack(level=-1).fillna(0)
df.columns = df.columns.droplevel()
df

,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7
Li-ion batteries,24.509671,4.298765,0.000,20.881362,55.748229,21.625011,31.077998,25.398298
Pumped hydro storage,0.000000,9.011000,8.265,19.620000,3.889000,4.018000,9.765000,10.605000
Salt caverns (hydrogen),1197.000000,0.000000,11571.000,4256.000000,5985.000000,266.000000,0.000000,0.000000
Salt caverns (methane),0.000000,0.000000,0.000,1772.000000,2215.000000,0.000000,0.000000,0.000000


In [33]:
df = pd.DataFrame(esM._pyM.cap_trans.get_values(), index=[0]).T.swaplevel(i=0, j=2, axis=0).swaplevel(i=1, j=2, axis=0).sort_index()
df = df.unstack(level=-1).fillna(0)
df.columns = df.columns.droplevel()
df

cluster_0  cluster_1  cluster_2  cluster_3  \
AC cables           cluster_0   0.000000   0.000000   0.000000   2.395773   
                    cluster_1   0.000000   0.000000   8.905339   4.791545   
                    cluster_2   0.000000   8.905339   0.000000   0.000000   
                    cluster_3   2.395773   4.791545   0.000000   0.000000   
                    cluster_4   3.262664   0.000000   8.731961   5.737245   
                    cluster_5   2.395773   0.000000   0.000000  13.381651   
                    cluster_6   0.000000   0.000000   0.000000   0.945700   
                    cluster_7   0.000000   9.866801   0.000000   3.593659   
DC cables           cluster_0   0.000000   2.000000   0.000000   0.000000   
                    cluster_1   2.000000   0.000000   2.000000   0.000000   
                    cluster_2   0.000000   2.000000   0.000000   0.000000   
                    cluster_3   0.000000   0.000000   0.000000   0.000000   
                    cluster_4   0.000000   0.000000   2.000000   0.000000   
                    cluster_7   2.000000   0.000000   0.000000   4.000000   
Pipeline (hydrogen) cluster_0   0.000000   0.000000   0.000000   0.000000   
                    cluster_1   0.000000   0.000000   7.143182   0.000000   
                    cluster_2   0.000000   7.143182   0.000000   0.000000   
                    cluster_3   0.000000   0.000000   0.000000   0.000000   
                    cluster_4  13.393819   0.000000  12.608102   4.284904   
                    cluster_5   0.000000   0.000000   0.000000   1.619352   
                    cluster_6   0.000000   0.000000   0.000000   0.000000   
                    cluster_7   0.000000   3.535599   0.000000   0.000000   
Pipelines (methane) cluster_0   0.000000   0.000000   0.000000   0.000000   
                    cluster_1   0.000000   0.000000   2.733284   0.000000   
                    cluster_2   0.000000   2.733284   0.000000   0.000000   
                    cluster_3   0.000000   0.000000   0.000000   0.000000   
                    cluster_4   0.950972   0.000000   3.567567   2.107657   
                    cluster_5   0.000000   0.000000   0.000000   1.406284   
                    cluster_6   0.000000   0.000000   0.000000   0.000000   
                    cluster_7   0.000000   1.874250   0.000000   0.000000   

                               cluster_4  cluster_5  cluster_6  cluster_7  
AC cables           cluster_0   3.262664   2.395773   0.000000   0.000000  
                    cluster_1   0.000000   0.000000   0.000000   9.866801  
                    cluster_2   8.731961   0.000000   0.000000   0.000000  
                    cluster_3   5.737245  13.381651   0.945700   3.593659  
                    cluster_4   0.000000   0.000000   0.000000   0.000000  
                    cluster_5   0.000000   0.000000   1.796830   0.000000  
                    cluster_6   0.000000   1.796830   0.000000   6.178572  
                    cluster_7   0.000000   0.000000   6.178572   0.000000  
DC cables           cluster_0   0.000000   0.000000   0.000000   2.000000  
                    cluster_1   0.000000   0.000000   0.000000   0.000000  
                    cluster_2   2.000000   0.000000   0.000000   0.000000  
                    cluster_3   0.000000   0.000000   0.000000   4.000000  
                    cluster_4   0.000000   0.000000   0.000000   0.000000  
                    cluster_7   0.000000   0.000000   0.000000   0.000000  
Pipeline (hydrogen) cluster_0  13.393819   0.000000   0.000000   0.000000  
                    cluster_1   0.000000   0.000000   0.000000   3.535599  
                    cluster_2  12.608102   0.000000   0.000000   0.000000  
                    cluster_3   4.284904   1.619352   0.000000   0.000000  
                    cluster_4   0.000000   0.000000   0.000000   0.000000  
                    cluster_5   0.000000   0.000000   0.000000   0.000000  
                    cluster_6   0.000000   0.00